## LangChain Fundamentals: Step-by-Step Practice with Prompts,Embeddings, Tools, RAG, and Agents Using Gemini 1.5 Flash


In [ ]:
import os, getpass

if " " not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

Enter your Google AI API key: ··········


In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")  # fast, inexpensive model
resp = llm.invoke("Say hello from LangChain in one short sentence.")
print(resp.content)


Hello from LangChain!


In [3]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.7,
    max_tokens=256,
)

# Simple usage
response = llm.invoke("What are the benefits of renewable energy?")
print(response.content)        # Text output
print(response.usage_metadata) # Optional: token usage details

Renewable energy sources offer a multitude of benefits compared to fossil fuels:

**Environmental Benefits:**

* **Reduced Greenhouse Gas Emissions:** This is arguably the most significant benefit.  Renewable energy sources produce little to no greenhouse gases during operation, significantly reducing our carbon footprint and mitigating climate change.
* **Improved Air Quality:**  Burning fossil fuels releases harmful pollutants like particulate matter, sulfur dioxide, and nitrogen oxides, which contribute to respiratory illnesses and other health problems. Renewable energy sources drastically reduce these pollutants, leading to cleaner air.
* **Reduced Water Pollution:** Fossil fuel extraction and processing often contaminate water sources. Renewable energy technologies generally have a much lower impact on water quality.
* **Protection of Ecosystems:**  Renewable energy sources require less land disruption than fossil fuel extraction, helping to protect natural habitats and biodivers

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage

chat_model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.3,
    max_tokens=256,
)

messages = [
    SystemMessage(content="You are a helpful AI assistant."),
    HumanMessage(content="Explain quantum computing in simple terms."),
]

response = chat_model.invoke(messages)

print(response.content)



Imagine a light switch. It can be either ON or OFF, right?  That's how regular computers work – they use bits, which are either 0 or 1.

Quantum computers are different. They use **qubits**.  A qubit can be 0, 1, or *both at the same time*! This "both at the same time" is called **superposition**.  It's like the light switch being both ON and OFF simultaneously – weird, but true.

Another quantum trick is **entanglement**. Imagine two qubits linked together magically.  If you measure one and it's 0, you instantly know the other is 1, no matter how far apart they are!

Because of superposition and entanglement, quantum computers can explore many possibilities at once.  Think of it like searching a maze: a regular computer tries each path one by one, while a quantum computer explores all paths simultaneously.  This makes them potentially much faster for certain types of problems.

However, quantum computers aren't meant to replace regular computers. They're good at specific tasks, like:


In [4]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(
    model="gemini-embedding-001"
)

text = "LangChain is a powerful framework for LLM applications."
embedding_vector = embeddings.embed_query(text)

print(f"Embedding dimension: {len(embedding_vector)}")
print(f"First 5 values: {embedding_vector[:5]}")


Embedding dimension: 3072
First 5 values: [-0.011009139940142632, 0.003710047109052539, 0.007905430160462856, -0.07840708643198013, -0.012440552935004234]


In [5]:
from langchain_core.prompts import PromptTemplate

template = """
You are an expert {expertise} consultant.
Please provide advice on: {query}
Consider the following context: {context}
Your advice:
"""

prompt = PromptTemplate(
    input_variables=["expertise", "query", "context"],
    template=template,
)

formatted_prompt = prompt.format(
    expertise="financial planning",
    query="retirement savings strategies",
    context="for a 30-year-old software engineer earning $120k annually",
)
print("----- Formatted Prompt -----")
print(formatted_prompt)

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.3,
    max_tokens=300,
)

response = llm.invoke(formatted_prompt)
print("\n----- Model Response -----")
print(response.content)

----- Formatted Prompt -----

You are an expert financial planning consultant.
Please provide advice on: retirement savings strategies
Consider the following context: for a 30-year-old software engineer earning $120k annually
Your advice:


----- Model Response -----
## Retirement Savings Strategies for a 30-Year-Old Software Engineer Earning $120k Annually

Congratulations on your successful career as a software engineer!  At 30, you're in an excellent position to build a robust retirement plan.  However, simply saving is insufficient; a strategic approach is crucial.  Here's a comprehensive plan tailored to your situation:

**I. Maximizing Employer-Sponsored Retirement Plans:**

* **401(k) or Similar Plan:**  Your highest priority should be maximizing contributions to your employer-sponsored retirement plan.  If your employer offers a matching contribution (e.g., 50% up to 6% of your salary), contribute at least that much – this is essentially free money.  Aim to contribute the maxim

In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.chat import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

# Create chat prompt template
system_template = "You are a {role} who {specialty}."
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

human_template = "{request}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([
    system_message_prompt,
    human_message_prompt,
])

# Format the messages
messages = chat_prompt.format_messages(
    role="data scientist",
    specialty="specializes in machine learning model optimization",
    request="How can I improve the performance of my neural network?"
)

for message in messages:
    print(f"{message.__class__.__name__}: {message.content}")


SystemMessage: You are a data scientist who specializes in machine learning model optimization.
HumanMessage: How can I improve the performance of my neural network?


In [9]:
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate

# Define examples
examples = [
    {
        "input": "I need to analyze sales data",
        "output": "I recommend using pandas for data manipulation, matplotlib for visualization, and seaborn for statistical plots."
    },
    {
        "input": "How do I build a web scraper?",
        "output": "For web scraping, use requests for HTTP, BeautifulSoup for HTML parsing, and optionally Selenium for dynamic pages."
    }
]

# Create example template
example_template = """
Input: {input}
Output: {output}
""".strip()

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template=example_template,
)

# Create few-shot prompt
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="You are a Python programming expert. Provide helpful library recommendations:",
    suffix="Input: {query}\nOutput:",
    input_variables=["query"],
    example_separator="\n\n",
)

# Use the prompt
formatted = few_shot_prompt.format(query="I want to build a REST API for my project")
print(formatted)


You are a Python programming expert. Provide helpful library recommendations:

Input: I need to analyze sales data
Output: I recommend using pandas for data manipulation, matplotlib for visualization, and seaborn for statistical plots.

Input: How do I build a web scraper?
Output: For web scraping, use requests for HTTP, BeautifulSoup for HTML parsing, and optionally Selenium for dynamic pages.

Input: I want to build a REST API for my project
Output:


In [10]:
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser

# Create components
prompt = PromptTemplate(
    input_variables=["topic"],
    template="Write a brief summary about {topic}:"
)

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.7,
)

# Create chain (prompt -> model -> string)
chain = prompt | llm | StrOutputParser()

# Run chain
result = chain.invoke({"topic": "artificial intelligence"})
print(result)


Artificial intelligence (AI) is a broad field encompassing the development of computer systems capable of performing tasks that typically require human intelligence.  These tasks include learning, reasoning, problem-solving, perception, and natural language understanding.  AI systems achieve this through various techniques, including machine learning (where systems learn from data without explicit programming) and deep learning (a subset of machine learning using artificial neural networks with multiple layers).  AI is rapidly transforming many industries, from healthcare and finance to transportation and entertainment, offering both immense potential and significant ethical considerations.


In [ ]:
from langchain_community.document_loaders import PyPDFLoader

# Load a PDF file
loader = PyPDFLoader("Demo Setup Diagram.pdf")
pages = loader.load()

print(f"Loaded {len(pages)} pages")
for i, page in enumerate(pages[:3]):  # Show first 3 pages
    print(f"Page {i + 1}: {page.page_content[:150]}...")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 1.1 MB/s eta 0:00:00
Loaded 2 pages
Page 1: Here’s a demo setup diagram for your Smart Energy Optimization System:
🔹 Demo Setup Diagram
        ┌───────────────────────────┐
        │        Sen...
Page 2: This diagram is simple, clear , and examiner-friendly. ✅
2...


In [18]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate(
    input_variables=["topic"],
    template="Write a two-sentence overview about {topic}:"
)

chain = prompt | llm | StrOutputParser()
print(chain.invoke({"topic": "retrieval-augmented generation"}))


Retrieval-augmented generation (RAG) combines large language models (LLMs) with external knowledge sources, allowing the model to access and incorporate relevant information before generating a response.  This approach improves the accuracy, factual consistency, and overall quality of the generated text by grounding the LLM's output in specific evidence.


In [19]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Keep answers concise."),
    ("human", "{question}")
])

print((chat_prompt | llm | StrOutputParser()).invoke({"question": "What is LangChain?"}))


LangChain is a framework for developing applications powered by language models.


In [20]:
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from langchain_core.prompts import PromptTemplate

class Note(BaseModel):
    title: str = Field(..., description="Short title")
    bullets: list[str] = Field(..., description="3 concise bullet points")

parser = JsonOutputParser(pydantic_object=Note)

tpl = PromptTemplate(
    template="Create study notes as JSON about: {topic}\n{format_instructions}",
    input_variables=["topic"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

json_chain = tpl | llm | parser
print(json_chain.invoke({"topic": "vector databases"}))


{'title': 'Vector Databases', 'bullets': ['Store data as vectors, enabling similarity search based on semantic meaning rather than exact keyword matches.', 'Ideal for applications needing to find similar items, such as image search, recommendation systems, and anomaly detection.', 'Utilize techniques like approximate nearest neighbor (ANN) search to efficiently handle large datasets and high-dimensional vectors.']}


In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

memory = ConversationBufferMemory(return_messages=True)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are friendly. Keep replies under 2 sentences."),
    MessagesPlaceholder("history"),
    ("human", "{input}")
])

def chat_turn(text):
    hist = memory.load_memory_variables({})["history"]
    out = (prompt | llm | StrOutputParser()).invoke({"history": hist, "input": text})
    memory.save_context({"input": text}, {"output": out})
    return out

print(chat_turn("My name is Chandan."))
print(chat_turn("What did I say my name is?"))

It's nice to meet you, Chandan!  How can I help you today?
You said your name is Chandan.


In [27]:
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

docs = [
    Document(page_content="LangChain provides prompts, chains, memory, and tools."),
    Document(page_content="RAG retrieves context before generation for grounded answers."),
    Document(page_content="FAISS enables fast vector similarity search."),
]

emb = GoogleGenerativeAIEmbeddings(model="gemini-embedding-001")
vs = FAISS.from_documents(docs, emb)
retriever = vs.as_retriever(search_kwargs={"k": 2})

rag_prompt = PromptTemplate.from_template(
    "Use the context to answer the question.\n\nContext:\n{ctx}\n\nQuestion: {q}\n\nAnswer briefly:"
)

def join(ds): return "\n\n".join(d.page_content for d in ds)

rag_chain = ({"ctx": retriever | join, "q": RunnablePassthrough()} | rag_prompt | llm | StrOutputParser())
print(rag_chain.invoke("What does FAISS do in RAG?"))


FAISS enables fast similarity search for context retrieval in RAG.


In [29]:
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "Be concise."),
    ("human", "Explain retrieval-augmented generation in 5 lines.")
])
for chunk in (chat_prompt | llm | StrOutputParser()).stream({}):
    print(chunk, end="", flush=True)
print()


Retrieval-augmented generation (RAG) combines large language models (LLMs) with external knowledge sources.  It retrieves relevant information before generating text. This improves accuracy and reduces hallucinations.  The retrieved context is often included in the output.  RAG enhances LLM capabilities by grounding responses in factual data.



In [30]:
validator = PromptTemplate.from_template(
    "Is the following text 2-3 sentences and formal tone? Answer yes/no.\n\n{text}"
)
def ensure_constraints(topic):
    text = chain.invoke({"topic": topic})
    verdict = (validator | llm | StrOutputParser()).invoke({"text": text}).lower()
    if "yes" in verdict:
        return text
    fix = PromptTemplate.from_template(
        "Rewrite to 2-3 sentences, formal tone:\n\n{text}"
    ) | llm | StrOutputParser()
    return fix.invoke({"text": text})

print(ensure_constraints("contrast RAG vs fine-tuning"))


Retrieval Augmented Generation (RAG) uses a separate knowledge base to retrieve relevant information for context before generating a response, maintaining the model's general capabilities.  Fine-tuning, conversely, directly modifies the model's weights with task-specific data, improving performance on that task but potentially reducing its adaptability to other tasks.


In [31]:
docs = [
    "LangChain composes prompts, models, and tools.",
    "RAG retrieves relevant chunks before generation.",
    "Vector DBs enable semantic search over embeddings."
]

map_prompt = ChatPromptTemplate.from_messages([
    ("human", "Write a one-sentence summary:\n\n{context}")
])
map_chain = map_prompt | llm | StrOutputParser()
mapped = map_chain.batch([{"context": d} for d in docs])

reduce_prompt = ChatPromptTemplate.from_messages([
    ("human", "Combine these summaries into one paragraph:\n\n{docs}")
])
reduce_chain = reduce_prompt | llm | StrOutputParser()
final_summary = reduce_chain.invoke({"docs": "\n".join(f"- {m}" for m in mapped)})
print(final_summary)


LangChain is a framework for developing applications that leverage the power of large language models (LLMs).  These applications are often enhanced through Retrieval Augmented Generation (RAG) systems, which improve LLM output by integrating information retrieved from external sources.  This retrieval process is frequently facilitated by vector databases, which efficiently store and search data based on semantic meaning via embeddings, enabling more accurate and contextually relevant responses.
